# Load Dataset's

In [74]:
import pandas as pd

# load the data into a Pandas DataFrame
users = pd.read_csv('users.csv')
follows = pd.read_csv('follows.csv')

In [75]:
users.head(3)

,user_id,first_name,last_name,house
0,1,Harry,Potter,Gryffindor
1,2,Ron,Wesley,Gryffindor
2,3,Hermonie,Granger,Gryffindor


In [76]:
follows.head(3)

,user_id,follows,date
0,1,2,1993-09-01
1,2,1,1989-01-01
2,3,1,1993-07-01


In [78]:
follows[1:5]

,user_id,follows,date
1,2,1,1989-01-01
2,3,1,1993-07-01
3,2,3,1994-10-10
4,3,2,1995-03-01


## How many users are there in each house? 

SQL querie:

In [ ]:
SELECT house, COUNT(house) AS user_count 
FROM users AS u
GROUP BY u.house

In [ ]:
house     |USER_COUNT|
----------+----------+
Gryffindor|         4|
Slytherin |         2|
Ravenclaw |         2|
Hufflepuff|         1|

Pandas:

In [16]:
users['house'].value_counts()

Gryffindor    4
Slytherin     2
Ravenclaw     2
Hufflepuff    1
Name: house, dtype: int64

In [14]:
users.groupby('house').count()

,user_id,first_name,last_name
house,,,
Gryffindor,4,4,4
Hufflepuff,1,1,1
Ravenclaw,2,2,2
Slytherin,2,2,2


## List all following links that were created before September 1st, 1993 

SQL querie:

In [ ]:
SELECT * 
FROM follows AS f
WHERE f.date < '1993-09-01'
ORDER BY f.date 

In [ ]:
user_id|follows|date      |
-------+-------+----------+
5      |6      |1986-01-10|
4      |2      |1988-08-08|
4      |1      |1988-08-08|
2      |1      |1989-01-01|
7      |1      |1990-02-02|
3      |1      |1993-07-01|

Pandas:

In [29]:
before_19930901 = follows[follows.date < '1993-09-01'].sort_values(by=['date'])

In [30]:
before_19930901

,user_id,follows,date
10,5,6,1986-01-10
5,4,2,1988-08-08
6,4,1,1988-08-08
1,2,1,1989-01-01
11,7,1,1990-02-02
2,3,1,1993-07-01


## List all the following links established before September 1st 1993, but this time use the users first names

SQL querie:

In [ ]:
SELECT follows.date, users.first_name 
FROM follows 
LEFT JOIN users ON users.user_id = follows.user_id
WHERE follows.date < '1993-09-01'
ORDER BY follows.date 

In [ ]:
date      |first_name|
----------+----------+
1986-01-10|Draco     |
1988-08-08|Ginny     |
1988-08-08|Ginny     |
1989-01-01|Ron       |
1990-02-02|Luna      |
1993-07-01|Hermonie  |

Pandas:

In [69]:
df_merged = follows.merge(users, how='left', on='user_id') 

In [70]:
df_merged_filter = df_merged[['date', 'first_name']]

In [71]:
df_merged_filter.head(3)

,date,first_name
0,1993-09-01,Harry
1,1989-01-01,Ron
2,1993-07-01,Hermonie


In [72]:
before_19930901_2 = df_merged_filter[df_merged_filter.date < '1993-09-01'].sort_values(by=['date'])

In [73]:
before_19930901_2

,date,first_name
10,1986-01-10,Draco
5,1988-08-08,Ginny
6,1988-08-08,Ginny
1,1989-01-01,Ron
11,1990-02-02,Luna
2,1993-07-01,Hermonie


## Give a count of how many people followed each user as of 1999-12-31. Give the result in term of "users full name, number of followers

SQL querie:

In [ ]:
SELECT 
    u.first_name || ' ' || u.last_name AS Full_name,
    COUNT(f.follows) AS Count_followers
FROM follows AS f
JOIN users AS u ON u.user_id = f.follows 
WHERE f.date < '1999-12-31'
GROUP BY f.follows

In [ ]:
Full_name       |Count_followers|
----------------+---------------+
Harry Potter    |              5|
Ron Wesley      |              3|
Hermonie Granger|              1|
Ginny Weasley   |              1|
Tom Riddle      |              1|
Luna Lovegood   |              1|
Cho Chang       |              3|
Cedric Diggory  |              3|

Pandas:

In [123]:
df_foll = df_merged[df_merged.date < '1999-12-31'].groupby('follows', as_index=False).count()
df_foll = df_foll[['follows', 'date']]

In [124]:
df_foll = df_foll.rename(columns={'follows': 'user_id', 'date': 'Count followers'})

In [125]:
df_foll = df_foll.merge(users, how='left', on='user_id') 

In [126]:
df_foll["Full name"] = df_foll.first_name + " " + df_foll.last_name

In [128]:
df_foll = df_foll[['Full name', 'Count followers']]

In [129]:
df_foll

,Full name,Count followers
0,Harry Potter,5
1,Ron Wesley,3
2,Hermonie Granger,1
3,Ginny Weasley,1
4,Tom Riddle,1
5,Luna Lovegood,1
6,Cho Chang,3
7,Cedric Diggory,3


## List the number of users each user follows

SQL querie:

In [ ]:
SELECT 
    u.first_name || ' ' || u.last_name AS Full_name,
    COUNT(f.user_id) AS Count_people_followed
FROM follows AS f
JOIN users AS u ON u.user_id = f.user_id 
GROUP BY f.user_id 

In [ ]:
Full_name       |Count_people_followed|
----------------+---------------------+
Harry Potter    |                    5|
Ron Wesley      |                    2|
Hermonie Granger|                    3|
Ginny Weasley   |                    3|
Draco Malfoy    |                    2|
Luna Lovegood   |                    2|
Cho Chang       |                    2|
Cedric Diggory  |                    1|

Pandas:

In [179]:
df_user = df_merged.groupby('user_id', as_index=False).count()

In [180]:
df_user = df_user[['user_id', 'follows']]

In [181]:
df_user = df_user.merge(users, how='left', on='user_id') 

In [182]:
df_user["Full name"] = df_user.first_name + " " + df_user.last_name

In [183]:
df_user = df_user[['Full name', 'follows']]

In [184]:
df_user = df_user.rename(columns={'follows': 'Count people followed'})

In [185]:
df_user

,Full name,Count people followed
0,Harry Potter,5
1,Ron Wesley,2
2,Hermonie Granger,3
3,Ginny Weasley,3
4,Draco Malfoy,2
5,Luna Lovegood,2
6,Cho Chang,2
7,Cedric Diggory,1


## List all rows from follows where someone from one house follows someone from a different house. Please give the result with user names. 

SQL querie:

Pandas:

In [220]:
df_houses = df_merged
df_houses["Full name user"] = df_houses.first_name + " " + df_houses.last_name
df_houses = df_houses[['Full name user', 'house', 'follows']]

In [221]:
df_houses = df_houses.rename(columns={'house': 'House of user'})

In [222]:
df_houses = pd.merge(df_houses,users, how='left',left_on=['follows'],right_on=['user_id'])

In [223]:
df_houses["Full name follower"] = df_houses.first_name + " " + df_houses.last_name

In [224]:
df_houses = df_houses.rename(columns={'house': 'House of follower'})

In [225]:
df_houses = df_houses[['Full name user', 'House of user', 'Full name follower', 'House of follower']]

In [226]:
df_houses_dif = df_houses[df_houses['House of user'] != df_houses['House of follower']]

In [227]:
df_houses_dif[['Full name user']]

,Full name user
8,Harry Potter
9,Draco Malfoy
11,Luna Lovegood
12,Harry Potter
13,Harry Potter
14,Cho Chang
15,Cho Chang
16,Cedric Diggory
18,Hermonie Granger
19,Ginny Weasley


## List all unrequited followings (i.e. where A follows B but B does not follow A) 

SQL querie:

Pandas:

In [228]:
df_houses

,Full name user,House of user,Full name follower,House of follower
0,Harry Potter,Gryffindor,Ron Wesley,Gryffindor
1,Ron Wesley,Gryffindor,Harry Potter,Gryffindor
2,Hermonie Granger,Gryffindor,Harry Potter,Gryffindor
3,Ron Wesley,Gryffindor,Hermonie Granger,Gryffindor
4,Hermonie Granger,Gryffindor,Ron Wesley,Gryffindor
5,Ginny Weasley,Gryffindor,Ron Wesley,Gryffindor
6,Ginny Weasley,Gryffindor,Harry Potter,Gryffindor
7,Harry Potter,Gryffindor,Ginny Weasley,Gryffindor
8,Harry Potter,Gryffindor,Draco Malfoy,Slytherin
9,Draco Malfoy,Slytherin,Harry Potter,Gryffindor
